In [1]:
import os
import numpy as np
from collections import defaultdict
import xml.etree.ElementTree as ET
from collections import Counter

# filename
DATA_DIR = '../data'
TRAIN_ANS_FILE = os.path.join(DATA_DIR, 'ans_train.csv')
QUERY_TRAIN_FILE = os.path.join(DATA_DIR, 'query-train.xml')
QUERY_TEST_FILE = os.path.join(DATA_DIR, 'query-test.xml')
FILE_LIST = os.path.join(DATA_DIR, 'file-list')
INV_FILE = os.path.join(DATA_DIR, 'inverted-file')
VOCAB_FILE = os.path.join(DATA_DIR, 'vocab.all')
OUTPUT_FILE = 'output.csv'

# building maps
docname2id = dict()
with open(FILE_LIST, 'r') as f:
    for idx, line in enumerate(f):
        docname2id[line.strip()] = idx
id2docname = {y:x for x, y in docname2id.items()}

word2id = dict()
with open(VOCAB_FILE, 'r') as f:
    f.readline()
    for idx, line in enumerate(f, 1):
        word2id[line.strip()] = idx

word_freq = dict()
gram_freq = dict()
gram2id = dict()
with open(INV_FILE, 'r') as f:
    idx = 0
    while True:
        line = f.readline().strip()
        if not line:
            break
        id_1, id_2, doc_count = [int(i) for i in line.split(' ')]
        doc_records = defaultdict(int)
        for i in range(doc_count):
            doc_id, freq = [int(i) for i in f.readline().strip().split(' ')]
            doc_records[doc_id] = freq
        if id_2 == -1:
            word_freq[(id_1)] = doc_records
#         else:
#             gram_freq[(id_1, id_2)] = doc_records
#             gram2id[(id_1, id_2)] = idx
#             idx += 1
        gram_freq[(id_1, id_2)] = doc_records
        gram2id[(id_1, id_2)] = idx
        idx += 1

In [2]:
# doc2len = defaultdict(int)
# for word_id, records in word_freq.items():
#     for doc_id, freq in records.items():
#         doc2len[doc_id] += freq
# avdl = sum([length for _, length in doc2len.items()]) / len(doc2len)
doc2len = defaultdict(int)
for docname, idx in docname2id.items():
    doc2len[idx] = os.path.getsize(os.path.join(DATA_DIR, docname))
avdl = sum([length for _, length in doc2len.items()]) / len(doc2len)

In [3]:
tree = ET.ElementTree(file=QUERY_TRAIN_FILE)
root = tree.getroot()

query_num = len(root)
word_num = len(gram2id) + 1
doc_num = len(docname2id)

In [4]:
from scipy.sparse import dok_matrix

def two_char_to_gramid(c1, c2):
    try:
        c1 = word2id[c1]
        if c2 is None:
            c2 = -1
        else:
            c2 = word2id[c2]
        return gram2id[(c1, c2)]
    except:
        return 0

def generate_candidates_weight(k_1=1.5, b=0.75):
    weight = dok_matrix((doc_num, word_num), dtype=np.float32)
    for idx, (gram, records) in enumerate(gram_freq.items(), 1):
        word_id = gram2id[gram]
        IDF = np.log((doc_num - len(records) + 0.5) / (len(records) + 0.5))
        for doc_id, freq in records.items():
            TF = (k_1 + 1) * freq / (freq + k_1 * (1 - b + b * doc2len[doc_id] / avdl))
            weight[doc_id, word_id] = TF * IDF
        if idx % 1000 == 0:
            print('Has indexed %d grams' % idx)
    return weight

def generate_queries_weight(ka=100):
#     queries = dok_matrix((query_num, word_num), dtype=np.float32)
    queries = np.zeros((query_num, word_num))
    for query_id, child in enumerate(root):
        query = child[4].text.strip('\n。 ').split('、')
        grams = []
        for w in query:
            if len(w) == 1:
                grams.append(two_char_to_gramid(w, None))
                break
            else:
                for c in w:
                    grams.append(two_char_to_gramid(c, None))
                for ci, cj in zip(w[:-1], w[1:]):
                    grams.append(two_char_to_gramid(ci, cj))
#         print(grams)
        for word, freq in Counter(grams).items():
            if not word:
                continue
            TF = (ka + 1) * freq / (freq + ka)
            queries[query_id, word] = TF
    return queries

In [5]:
def MAP(top100):
    AP = []
    with open(TRAIN_ANS_FILE, 'r') as f:
        f.readline()
        for line, rank in zip(f, top100):
            idx, answer = line.strip().split(',')
            answer = set(answer.split())
            rank = [(id2docname[i].split('/')[-1].lower()) for i in rank]
            hit = 0
            P = []
            for rank_i, rank in enumerate(rank, 1):
                if rank in answer:
                    hit += 1
                    P.append(hit / rank_i)
            AP.append(sum(P) / len(P))
    return sum(AP) / len(AP)

In [ ]:
candidates = generate_candidates_weight(k_1=1.6, b=0.75)

In [ ]:
queries = generate_queries_weight(ka=5.0)

In [ ]:
# ret = np.matmul(queries, np.transpose(candidates)).argsort(axis=1)[:, ::-1][:, :100]
scores = candidates.dot(np.transpose(queries))
# print('k_1: %.2f, ka: %.2f, MPA: %.8f' % (k_1, ka, MAP(ret)))

In [ ]:
scores = np.transpose(scores)

In [ ]:
ret = scores.argsort(axis=1)[:, ::-1][:, :100]
MAP(ret)

In [ ]:
for k_1 in [1, 1.2, 1.4, 1.6, 1.8, 2.0]:
    for ka in [0, 5, 10, 50, 100, 300, 500, 1000]:
        candidates = generate_candidates_weight(k_1, b=0.75)
        queries = generate_queries_weight(ka)
        ret = np.matmul(queries, np.transpose(candidates)).argsort(axis=1)[:, ::-1][:, :100]
        print('k_1: %.2f, ka: %.2f, MPA: %.8f' % (k_1, ka, MAP(ret)))

In [6]:
# Testing Part
tree = ET.ElementTree(file=QUERY_TEST_FILE)
root = tree.getroot()
query_num = len(root)

candidates = generate_candidates_weight(k_1=1.6, b=0.75)
queries = generate_queries_weight(ka=5.0)
ret = (np.transpose(candidates.dot(np.transpose(queries)))).argsort(axis=1)[:, ::-1][:, :100]

with open(OUTPUT_FILE, 'w+') as f:
    print('query_id,retrieved_docs', file=f)
    for idx, result in enumerate(ret, 11):
        print(str(idx).zfill(3), ' '.join([(id2docname[i].split('/')[-1].lower()) for i in result]), sep=',', file=f)

Has indexed 1000 grams
Has indexed 2000 grams
Has indexed 3000 grams
Has indexed 4000 grams
Has indexed 5000 grams
Has indexed 6000 grams
Has indexed 7000 grams
Has indexed 8000 grams
Has indexed 9000 grams
Has indexed 10000 grams
Has indexed 11000 grams
Has indexed 12000 grams
Has indexed 13000 grams
Has indexed 14000 grams
Has indexed 15000 grams
Has indexed 16000 grams
Has indexed 17000 grams
Has indexed 18000 grams
Has indexed 19000 grams
Has indexed 20000 grams
Has indexed 21000 grams
Has indexed 22000 grams
Has indexed 23000 grams
Has indexed 24000 grams
Has indexed 25000 grams
Has indexed 26000 grams
Has indexed 27000 grams
Has indexed 28000 grams
Has indexed 29000 grams
Has indexed 30000 grams
Has indexed 31000 grams
Has indexed 32000 grams
Has indexed 33000 grams
Has indexed 34000 grams
Has indexed 35000 grams
Has indexed 36000 grams
Has indexed 37000 grams
Has indexed 38000 grams
Has indexed 39000 grams
Has indexed 40000 grams
Has indexed 41000 grams
Has indexed 42000 grams
H

Has indexed 334000 grams
Has indexed 335000 grams
Has indexed 336000 grams
Has indexed 337000 grams
Has indexed 338000 grams
Has indexed 339000 grams
Has indexed 340000 grams
Has indexed 341000 grams
Has indexed 342000 grams
Has indexed 343000 grams
Has indexed 344000 grams
Has indexed 345000 grams
Has indexed 346000 grams
Has indexed 347000 grams
Has indexed 348000 grams
Has indexed 349000 grams
Has indexed 350000 grams
Has indexed 351000 grams
Has indexed 352000 grams
Has indexed 353000 grams
Has indexed 354000 grams
Has indexed 355000 grams
Has indexed 356000 grams
Has indexed 357000 grams
Has indexed 358000 grams
Has indexed 359000 grams
Has indexed 360000 grams
Has indexed 361000 grams
Has indexed 362000 grams
Has indexed 363000 grams
Has indexed 364000 grams
Has indexed 365000 grams
Has indexed 366000 grams
Has indexed 367000 grams
Has indexed 368000 grams
Has indexed 369000 grams
Has indexed 370000 grams
Has indexed 371000 grams
Has indexed 372000 grams
Has indexed 373000 grams


Has indexed 663000 grams
Has indexed 664000 grams
Has indexed 665000 grams
Has indexed 666000 grams
Has indexed 667000 grams
Has indexed 668000 grams
Has indexed 669000 grams
Has indexed 670000 grams
Has indexed 671000 grams
Has indexed 672000 grams
Has indexed 673000 grams
Has indexed 674000 grams
Has indexed 675000 grams
Has indexed 676000 grams
Has indexed 677000 grams
Has indexed 678000 grams
Has indexed 679000 grams
Has indexed 680000 grams
Has indexed 681000 grams
Has indexed 682000 grams
Has indexed 683000 grams
Has indexed 684000 grams
Has indexed 685000 grams
Has indexed 686000 grams
Has indexed 687000 grams
Has indexed 688000 grams
Has indexed 689000 grams
Has indexed 690000 grams
Has indexed 691000 grams
Has indexed 692000 grams
Has indexed 693000 grams
Has indexed 694000 grams
Has indexed 695000 grams
Has indexed 696000 grams
Has indexed 697000 grams
Has indexed 698000 grams
Has indexed 699000 grams
Has indexed 700000 grams
Has indexed 701000 grams
Has indexed 702000 grams


Has indexed 992000 grams
Has indexed 993000 grams
Has indexed 994000 grams
Has indexed 995000 grams
Has indexed 996000 grams
Has indexed 997000 grams
Has indexed 998000 grams
Has indexed 999000 grams
Has indexed 1000000 grams
Has indexed 1001000 grams
Has indexed 1002000 grams
Has indexed 1003000 grams
Has indexed 1004000 grams
Has indexed 1005000 grams
Has indexed 1006000 grams
Has indexed 1007000 grams
Has indexed 1008000 grams
Has indexed 1009000 grams
Has indexed 1010000 grams
Has indexed 1011000 grams
Has indexed 1012000 grams
Has indexed 1013000 grams
Has indexed 1014000 grams
Has indexed 1015000 grams
Has indexed 1016000 grams
Has indexed 1017000 grams
Has indexed 1018000 grams
Has indexed 1019000 grams
Has indexed 1020000 grams
Has indexed 1021000 grams
Has indexed 1022000 grams
Has indexed 1023000 grams
Has indexed 1024000 grams
Has indexed 1025000 grams
Has indexed 1026000 grams
Has indexed 1027000 grams
Has indexed 1028000 grams
Has indexed 1029000 grams
Has indexed 1030000 